In [5]:
import pandas as pd

C:\Users\symonets.h\AppData\Local\Temp\ipykernel_30880\4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [6]:
input_df_transformed = pd.ExcelFile('../data/transformed_data05_03_24.xlsx')
input_dict_transformed = {}
for sheet_name in input_df_transformed.sheet_names[1:16]:
    input_dict_transformed[sheet_name] = input_df_transformed.parse(sheet_name)

In [7]:
input_dict_transformed.keys()

dict_keys(['MG', 'LEMS', 'WZM', 'CPEO', 'MMN', 'typical CIDP', 'CMT', 'DMD BMD', 'LGMD', 'FSHD', 'DM1 ', 'DM2', 'SMA 3-4', 'Pompe', "SBMA (choroba Kennedy'ego)"])

In [8]:
disease_cards = {}
for disease, features in input_dict_transformed.items():
    dict_= {}
    for key, value in  features.loc[:, ~features.columns.str.contains('^Unnamed')].to_dict('list').items():
        dict_[key] = list(set([i for i in value if i==i]))
    disease_cards[disease] = dict_
    # print({key: list(set([i for i in value if i==i]))})

In [9]:
disease_cards

{'MG': {'grupa objawów': ['niedowład kończyn górnych',
   'Objawy opuszkowe',
   'Objawy oczne',
   'niedowład kończyn dolnych'],
  'nazwa objawu': ['dysfagia',
   'zaburzenia mowy, dyzartria',
   'zaburzenia gryzienia / żucia pokarmów',
   'duszność (tylko przy zajęciu innych grup mięśni)',
   'proksymalny niedowład kończyn dolnych (osłaienie siły ud, problemy ze wstawaniem z krzesła, z wchodzeniem po schodach, problem ze wstaniem z przysiadu)',
   'ptoza (opadanie powiek)',
   'proksymalny niedowład kończyn górnych (osłaienie siły w ramionach, problemy z unoszeniem ramion)',
   'dwojenie obrazu '],
  'nasilenie w czasie': ['zmienne'],
  'dynamika objawów': ['okresy zaostrzeń (pogorszenia / nasilenia) i remisji (poprawy)',
   'zmienność w ciągu doby'],
  'cechy objawu': ['symetryczny', 'asymetryczny '],
  'wiek wystapienia pierwszych objawów': ['30-50  rż',
   '>50rż',
   '10-20rż',
   '20-30rż',
   '<10 rż'],
  'wywiad rodzinny - czy w rodzinie są lub były osoby z podobnymi objawami 

In [10]:
def predict_disease(patient_data, disease_data):
    # Initializing counters for each diseases
    disease_counter = [0]*len(disease_data)
    for key in patient_data.keys():
        for i, disease in enumerate(disease_data):
            if key in disease.keys():
                for item in patient_data[key]:
                    if item in disease[key]:
                        # Increase counter if patient's symptoms match disease's symptoms
                        disease_counter[i] += 1
    
    total_symptoms = sum(disease_counter)
    disease_probabilities = [round((count/total_symptoms)*100, 2) for count in disease_counter]
    
    # Predict 'grupa chorób'
    max_index = disease_counter.index(max(disease_counter))
    predicted_disease = disease_data[max_index]['grupa chorób']

    # If patient provided enough data, predict 'podgrupa chorób' and 'jednostka chorobowa'.
    # If not, return predicted 'grupa chorób' with probabilities.
    if set(['poziom ck', 'objawy obligatoryjne', 'objawy wykluczające', 'choroby współistniejące', 'cechy charakterystyczne i objawy współistniejące'])&(set(patient_data.keys())):
        return {'grupa chorób': predicted_disease, 'podgrupa chorób': disease_data[max_index]['podgrupa chorób'], 'jednostka chorobowa': disease_data[max_index]['jednostka chorobowa'], 'probabilities': disease_probabilities}
    else:
        return {'grupa chorób': predicted_disease, 'probabilities': disease_probabilities}

In [11]:
# ['poziom ck', 'objawy obligatoryjne', 'objawy wykluczające', 'choroby współistniejące', 'cechy charakterystyczne i objawy współistniejące']

In [12]:
patient_data = {
    'nazwa objawu': ['ptoza (opadanie powiek)'],
    'nasilenie w czasie': ['zmienne']
    # include more features here
}
patient_data2 = {
    'nazwa objawu': ['proksymalny niedowład kończyn górnych (osłaienie siły w ramionach, problemy z unoszeniem ramion)'],
    'nasilenie w czasie': ['stałe/postępujące'],
    'cechy objawu':['symetryczny'],
    'wiek wystapienia pierwszych objawów':['<10 rż'],
    'wywiad rodzinny - czy w rodzinie są lub były osoby z podobnymi objawami jak u pacjenta. jeśli tak to proszę wybrac pokrewiensto':['brat'],
    'objawy wykluczające':['dominujące zajęcie mięśni twarzy']
    # include more features here
}
patient_data3 = {
    'nazwa objawu': ['ptoza (opadanie powiek)', ''],
    'nasilenie w czasie': ['stałe/postępujące'],
    'dynamika objawów': ['postępujące w ciągu lat- dekad'],
    'cechy objawu':['symetryczny'],
    'wiek wystapienia pierwszych objawów':['30-50  rż'],
    'wywiad rodzinny - czy w rodzinie są lub były osoby z podobnymi objawami jak u pacjenta. jeśli tak to proszę wybrac pokrewiensto':['brat'],
    'objawy wykluczające':['dominujące zajęcie mięśni twarzy']
    #  include more features here
}

In [13]:
predict_disease(patient_data, list(disease_cards.values()))

{'grupa chorób': ['zaburzenia transmisji nerwowo-mięśniowej'],
 'probabilities': [22.22,
  22.22,
  22.22,
  11.11,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  11.11,
  0.0,
  0.0,
  11.11,
  0.0]}

In [14]:
predict_disease(patient_data2, list(disease_cards.values()))

{'grupa chorób': ['miopatie'],
 'podgrupa chorób': ['dystrofie'],
 'jednostka chorobowa': ['LGMD'],
 'probabilities': [5.17,
  6.9,
  6.9,
  3.45,
  3.45,
  6.9,
  6.9,
  8.62,
  10.34,
  6.9,
  8.62,
  5.17,
  6.9,
  6.9,
  6.9]}

In [15]:
predict_disease(patient_data3, list(disease_cards.values()))

{'grupa chorób': ['miopatie'],
 'podgrupa chorób': ['mitochondriopatie'],
 'jednostka chorobowa': ['CPEO'],
 'probabilities': [5.26,
  7.02,
  5.26,
  8.77,
  5.26,
  7.02,
  7.02,
  5.26,
  8.77,
  5.26,
  8.77,
  5.26,
  7.02,
  7.02,
  7.02]}